In [1]:
import praw
import regex as re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import sklearn
import nltk

In [ ]:
clientid = ''
client_secret = ''
user_agent = ""

In [3]:
reddit = praw.Reddit(
    client_id=clientid,
    client_secret=client_secret,
    user_agent=user_agent
)

In [4]:
print(reddit.read_only)
# Output: True

True


In [5]:
import re

def find_matched_terms(text, terms):
    matched = []
    for term in terms:
        # Word-boundary search (e.g., matches "lead" but not "leadership")
        pattern = rf'\b{re.escape(term)}\b'
        if re.search(pattern, text, flags=re.IGNORECASE):
            matched.append(term)
    return matched


In [6]:
tar_sub = 'DnD'
battery_terms = ['spells','weapons','class','feats']
time_filter = 'month'

In [7]:
data = []
seen_comments = set()

for term in battery_terms:
    for submission in reddit.subreddit(tar_sub).search(term, sort="top", time_filter=time_filter):
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            if comment.id in seen_comments:
                continue
            text = comment.body.lower()
            matched_terms = find_matched_terms(text, battery_terms)
            if matched_terms:
                data.append({
                    'comment_id': comment.id,
                    'author': str(comment.author),
                    'text': text,
                    'matched_terms': matched_terms,
                    'submission_id': submission.id,
                    'submission_title': submission.title
                })
                seen_comments.add(comment.id)

# Convert to DataFrame
df = pd.DataFrame(data)


In [8]:
df.head()

,comment_id,author,text,matched_terms,submission_id,submission_title
0,n27ms3n,it_all_falls_apart,i have a player who every session for the past...,[class],1lvmytn,No one reads the rules anymore
1,n27iui5,FatNinjaWalrus,i'd just like to say that i learned nearly all...,[feats],1lvmytn,No one reads the rules anymore
2,n27phse,BrokenMirrorMan,i swear when people complain about player min ...,[spells],1lvmytn,No one reads the rules anymore
3,n27bd3i,Selphie12,"i'm willing to admit, i'm one of those people,...",[feats],1lvmytn,No one reads the rules anymore
4,n27h67c,LucielFairy,as someone who has dm’d at many tables for one...,[class],1lvmytn,No one reads the rules anymore


In [9]:
df.to_csv('reddit_scrapes/monthly.csv')